In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import average_precision_score
from sklearn.metrics import precision_recall_curve
from sklearn.utils import resample
from imblearn.over_sampling import SMOTE
from sklearn.metrics import roc_auc_score
import scipy.stats
%matplotlib inline

In [21]:
dg = pd.read_csv('LRM_all_data.csv')
dg = dg.replace([np.inf, -np.inf], np.nan)

dg

,Unnamed: 0,uhid,dischargestatus,pulserate_SE,pulserate_DFA,pulserate_ADF,pulserate_Mean,pulserate_Var,ecg_resprate_SE,ecg_resprate_DFA,...,currentdateheight,tpn-tfl,mean_bp,sys_bp,dia_bp,abd_difference,stool_day_total,total_intake,typevalue_Antibiotics,typevalue_Inotropes
0,0,RNEH.0000011301,1,0.002212,1.613817,-1.977509,-394.117823,325867.798499,-0.000000,NaN,...,-999.000000,-1.378358e+07,-925.902449,-925.428639,-927.385850,NaN,0.882245,-13675982.90,-14324420.0,-14324541.0
1,1,RSHI.0000020326,0,0.015172,1.256702,-10.043233,70.163355,95901.623950,0.010486,1.263680,...,35.431324,-8.153299e+07,-927.187886,-925.618424,-927.858269,NaN,2.985271,-80184386.80,-85969930.0,-88980930.0
2,2,RNEH.0000008375,1,0.003033,1.498004,-2.960924,-181.511418,265077.685183,-0.000000,NaN,...,-999.000000,-1.829664e+07,-978.873990,-978.523298,-979.056848,NaN,0.312666,-18279954.00,-18416565.0,-18416565.0
3,3,RNEH.0000013713,1,0.009192,1.767304,-1.415621,-538.704290,308146.698034,NaN,NaN,...,-999.000000,-1.393545e+06,-773.090429,-769.134653,-775.215842,NaN,1.000000,-1369698.60,-1513485.0,-1513485.0
4,4,RSHI.0000021953,1,0.012028,1.375051,-3.356594,-28.716118,176636.263161,0.011205,1.442948,...,11.399872,-2.511144e+07,-907.016045,-904.231908,-907.383406,NaN,2.115461,-24825009.30,-27150640.0,-27332640.0
5,5,RNEH.0000013396,0,0.059440,1.185131,-14.614407,126.987859,34832.264342,-0.000000,NaN,...,-137.338112,-5.293966e+07,-996.679934,-996.641019,-996.691669,NaN,2.327584,-52718493.65,-54140805.0,-54140805.0
6,6,RNEH.0000012581,1,0.018602,1.292305,-2.387235,8.676119,143740.068523,-0.000000,NaN,...,34.000000,-1.682640e+07,-888.415791,-887.353183,-889.551849,NaN,2.004671,-16690609.35,-17787075.0,-17787135.0
7,7,RSHI.0000017471,1,0.031209,1.303742,-9.927936,127.989667,27782.073437,0.006844,1.351879,...,-152.735899,-2.461401e+07,-686.018172,-678.504935,-688.123499,NaN,0.410761,-24239975.96,-25266415.0,-25268226.0
8,8,RSHI.0000023452,0,0.012001,1.352361,-9.450499,76.259005,84569.236959,0.006547,1.472008,...,-72.356275,-8.713144e+07,-940.306646,-938.894783,-940.772614,NaN,1.653312,-85631321.57,-89415500.0,-94405500.0
9,9,RSHI.0000015691,1,0.010413,1.600400,-2.302049,-135.636451,232906.801165,0.009146,1.648675,...,-999.000000,-2.022915e+06,-920.040767,-918.726619,-920.592326,NaN,0.907434,-2018775.00,-2082915.0,-2082915.0


In [3]:
def logistic(X_train, X_test, y_train):
    model = LogisticRegression()
    model.fit(X_train,y_train)
    y_pred = model.predict(X_test) 
    return y_pred
dg.gestation

0     29.57
1     26.00
2     28.71
3     36.71
4     31.43
5     30.00
6     25.14
7     29.14
8     26.00
9     24.43
10    30.57
11    29.00
12    26.29
13    25.86
14    28.14
15    34.86
16    26.00
17    29.29
18    31.71
19    32.14
20    25.71
21    34.14
Name: gestation, dtype: float64

In [4]:
cont = ['ecg_resprate_ADF', 'ecg_resprate_DFA', 'ecg_resprate_Mean',
       'ecg_resprate_SE', 'ecg_resprate_Var', 'fio2_ADF', 'fio2_DFA',
       'fio2_Mean', 'fio2_SE', 'fio2_Var', 'heartrate_ADF', 'heartrate_DFA',
       'heartrate_Mean', 'heartrate_SE', 'heartrate_Var', 
       'pulserate_ADF', 'pulserate_DFA', 'pulserate_Mean', 'pulserate_SE',
       'pulserate_Var', 'spo2_ADF', 'spo2_DFA', 'spo2_Mean', 'spo2_SE',
       'spo2_Var']

In [5]:
X = dg[cont]
y = dg['dischargestatus']
X = X.fillna(X.mean())
X

,ecg_resprate_ADF,ecg_resprate_DFA,ecg_resprate_Mean,ecg_resprate_SE,ecg_resprate_Var,fio2_ADF,fio2_DFA,fio2_Mean,fio2_SE,fio2_Var,...,pulserate_ADF,pulserate_DFA,pulserate_Mean,pulserate_SE,pulserate_Var,spo2_ADF,spo2_DFA,spo2_Mean,spo2_SE,spo2_Var
0,-6.727032,1.443201,-999.000000,-0.000000,0.000000,-2.014195,1.970788,-845.898844,0.000182,135279.048339,...,-1.977509,1.613817,-394.117823,0.002212,325867.798499,-1.978452,1.688640,-424.039388,0.001901,294370.836340
1,-16.049085,1.263680,-85.855271,0.010486,117353.765413,-4.833580,1.575450,-693.086516,0.001406,218465.683410,...,-10.043233,1.256702,70.163355,0.015172,95901.623950,-10.107657,1.283612,11.659930,0.007851,85128.703978
2,-6.727032,1.443201,-999.000000,-0.000000,0.000000,-3.717289,1.585758,-999.000000,-0.000000,0.000000,...,-2.960924,1.498004,-181.511418,0.003033,265077.685183,-2.998581,1.566195,-218.366151,0.002877,241621.183753
3,-6.727032,1.443201,-999.000000,0.005966,0.000000,-3.717289,1.585758,-999.000000,0.002562,0.000000,...,-1.415621,1.767304,-538.704290,0.009192,308146.698034,-1.456402,1.927674,-572.133993,0.002564,264506.533201
4,-6.993218,1.442948,-260.054130,0.011205,223275.629563,-3.373766,1.641320,-855.856031,0.001468,125735.761400,...,-3.356594,1.375051,-28.716118,0.012028,176636.263161,-3.268808,1.354726,-53.634211,0.005695,143373.167148
5,-6.727032,1.443201,-999.000000,-0.000000,0.000000,-3.717289,1.585758,-999.000000,-0.000000,0.000000,...,-14.614407,1.185131,126.987859,0.059440,34832.264342,-14.633138,1.273730,62.614356,0.002343,30758.671465
6,-6.727032,1.443201,-999.000000,-0.000000,0.000000,-3.717289,1.585758,-999.000000,-0.000000,0.000000,...,-2.387235,1.292305,8.676119,0.018602,143740.068523,-2.422868,1.387623,-44.183375,0.001412,128710.686752
7,-10.641523,1.351879,-3.931231,0.006844,50708.162383,-2.843765,1.565542,-428.656013,0.003097,256566.645186,...,-9.927936,1.303742,127.989667,0.031209,27782.073437,-9.866479,1.370198,73.148441,0.003631,25051.479551
8,-8.808113,1.472008,-443.833831,0.006547,272892.579890,-3.717289,1.585758,-999.000000,-0.000000,0.000000,...,-9.450499,1.352361,76.259005,0.012001,84569.236959,-9.336166,1.410211,20.157556,0.002808,75337.547398
9,-2.683931,1.648675,-205.461391,0.009146,203459.183761,-3.717289,1.585758,-999.000000,0.002562,0.000000,...,-2.302049,1.600400,-135.636451,0.010413,232906.801165,-2.300415,1.636138,-165.891127,0.009176,217326.322439


In [6]:
def training(X,y):
    auc_roc = []
    training = []

    for i in range(25):
        try:
            X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                            test_size=0.3 
                                                            )
            # SMOTE
            sm = SMOTE(k_neighbors = 2)
            X_train_sm, y_train_sm = sm.fit_sample(X_train, y_train)
            y_smote = logistic(X_train_sm, X_test, y_train_sm)
            y_smote_1 = logistic(X_train_sm,X_train,y_train_sm)
            auc_roc.append(roc_auc_score(y_test,y_smote))
            training.append(roc_auc_score(y_train,y_smote_1))
            continue
        except Exception as e:
            print(e,"error")
            continue
    return auc_roc,training
        
        #except:
        #    continue

In [7]:
def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return round(m,3), round(m-h,3), round(m+h,3)

In [8]:
an = training(X,y)

/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


Expected n_neighbors <= n_samples,  but n_samples = 2, n_neighbors = 3 error


/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


Only one class present in y_true. ROC AUC score is not defined in that case. error


/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


In [9]:
c_a = mean_confidence_interval(an[0])

In [10]:
c_t = mean_confidence_interval(an[1])

In [11]:
fixed = ['gender', 'birthweight',
       'birthlength', 'birthheadcircumference', 'inout_patient_status', 'baby_type', 'central_temp',
       'apgar_onemin', 'apgar_fivemin', 'apgar_tenmin', 'motherage',
       'conception_type', 'mode_of_delivery', 'steroidname', 'numberofdose',
       'gestation']

In [12]:
X = dg[fixed]
y = dg['dischargestatus']
X = X.fillna(X.mean())

In [13]:
an = training(X,y)
f_a = mean_confidence_interval(an[0])
f_t = mean_confidence_interval(an[1])

Expected n_neighbors <= n_samples,  but n_samples = 2, n_neighbors = 3 error
Only one class present in y_true. ROC AUC score is not defined in that case. error
Expected n_neighbors <= n_samples,  but n_samples = 2, n_neighbors = 3 error
Expected n_neighbors <= n_samples,  but n_samples = 2, n_neighbors = 3 error


In [14]:
inter = [ 'central_temp',
       'currentdateheight', 'currentdateweight', 
       'new_ph', 'rbs', 'stool_day_total',
       'temp', 'total_intake', 'totalparenteralvolume',
'peep_DFA',
'peep_Mean',
'peep_SE',
'peep_Var',
'pip_ADF',
'pip_DFA',
'pip_Mean',
'pip_SE',
'pip_Var',
'map_ADF',
'map_DFA',
'map_Mean',
'map_SE',
'map_Var',
'tidalvol_ADF',
'tidalvol_DFA',
'tidalvol_Mean',
'tidalvol_SE',
'tidalvol_Var',
'minvol_ADF',
'minvol_DFA',
'minvol_Mean',
'minvol_SE',
'minvol_Var',
'ti_ADF',
'ti_DFA',
'ti_Mean',
'ti_SE',
'ti_Var',
'fio2_ADF',
'fio2_DFA',
'fio2_Mean',
'fio2_SE',
'fio2_Var',
'mean_bp',
'sys_bp',
'dia_bp']

In [15]:
X = dg[inter]
y = dg['dischargestatus']
X = X.fillna(X.mean())
an = training(X,y)
i_a = mean_confidence_interval(an[0])
i_t = mean_confidence_interval(an[1])
X

/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


Only one class present in y_true. ROC AUC score is not defined in that case. error
Only one class present in y_true. ROC AUC score is not defined in that case. error


/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


Expected n_neighbors <= n_samples,  but n_samples = 2, n_neighbors = 3 error
Only one class present in y_true. ROC AUC score is not defined in that case. error


/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


,central_temp,currentdateheight,currentdateweight,new_ph,rbs,stool_day_total,temp,total_intake,totalparenteralvolume,peep_DFA,...,ti_SE,ti_Var,fio2_ADF,fio2_DFA,fio2_Mean,fio2_SE,fio2_Var,mean_bp,sys_bp,dia_bp
0,-999.0,-999.000000,803.642313,-970.112618,-904.464966,0.882245,4.936354,-13675982.90,-1.371026e+07,1.136080,...,0.000571,124628.638956,-2.014195,1.970788,-845.898844,0.000182,135279.048339,-925.902449,-925.428639,-927.385850
1,-999.0,35.431324,1008.476356,-999.000000,-895.800101,2.985271,-17.931078,-80184386.80,-8.137969e+07,1.569696,...,-0.000000,0.000000,-4.833580,1.575450,-693.086516,0.001406,218465.683410,-927.187886,-925.618424,-927.858269
2,36.5,-999.000000,803.939571,-999.000000,-999.000000,0.312666,-999.000000,-18279954.00,-1.829149e+07,1.534177,...,-0.000000,0.000000,-3.717289,1.585758,-999.000000,-0.000000,0.000000,-978.873990,-978.523298,-979.056848
3,-999.0,-999.000000,2846.000000,-955.837769,-953.135314,1.000000,23.789571,-1369698.60,-1.381699e+06,1.534177,...,0.002363,0.000000,-3.717289,1.585758,-999.000000,0.002562,0.000000,-773.090429,-769.134653,-775.215842
4,-999.0,11.399872,796.845541,-996.609041,-862.999854,2.115461,-135.789554,-24825009.30,-2.493095e+07,1.636119,...,0.001362,120350.140976,-3.373766,1.641320,-855.856031,0.001468,125735.761400,-907.016045,-904.231908,-907.383406
5,-999.0,-137.338112,1201.895212,-995.379122,-986.236276,2.327584,28.523078,-52718493.65,-5.289264e+07,1.534177,...,-0.000000,0.000000,-3.717289,1.585758,-999.000000,-0.000000,0.000000,-996.679934,-996.641019,-996.691669
6,-999.0,34.000000,736.598332,-984.454756,-959.973534,2.004671,18.843503,-16690609.35,-1.674405e+07,1.534177,...,-0.000000,0.000000,-3.717289,1.585758,-999.000000,-0.000000,0.000000,-888.415791,-887.353183,-889.551849
7,-999.0,-152.735899,519.817174,-933.855003,-761.852842,0.410761,13.946777,-24239975.96,-2.432445e+07,1.565560,...,-0.000000,0.000000,-2.843765,1.565542,-428.656013,0.003097,256566.645186,-686.018172,-678.504935,-688.123499
8,-999.0,-72.356275,1069.056889,-998.307877,-902.854910,1.653312,-587.402133,-85631321.57,-8.692398e+07,1.534177,...,-0.000000,0.000000,-3.717289,1.585758,-999.000000,-0.000000,0.000000,-940.306646,-938.894783,-940.772614
9,-999.0,-999.000000,700.000000,-999.000000,-749.503597,0.907434,-23.091415,-2018775.00,-2.018775e+06,1.534177,...,0.002363,0.000000,-3.717289,1.585758,-999.000000,0.002562,0.000000,-920.040767,-918.726619,-920.592326


In [16]:
fixed_cont = list(set(fixed+cont))
X = dg[fixed_cont]
y = dg['dischargestatus']
X = X.fillna(X.mean())
an = training(X,y)
fc_a = mean_confidence_interval(an[0])
fc_t = mean_confidence_interval(an[1])

/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


In [17]:
fixed_inter = list(set(fixed+inter))
X = dg[fixed_inter]
y = dg['dischargestatus']
X = X.fillna(X.mean())
an = training(X,y)
fi_a = mean_confidence_interval(an[0])
fi_t = mean_confidence_interval(an[1])

/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


Only one class present in y_true. ROC AUC score is not defined in that case. error


/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


In [18]:
cont_inter = list(set(cont+inter))
X = dg[cont_inter]
y = dg['dischargestatus']
X = X.fillna(X.mean())
an = training(X,y)
ci_a = mean_confidence_interval(an[0])
ci_t = mean_confidence_interval(an[1])

/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


Only one class present in y_true. ROC AUC score is not defined in that case. error


/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


Only one class present in y_true. ROC AUC score is not defined in that case. error


/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


In [19]:
all_cols = list(set(cont+inter+fixed))
X = dg[all_cols]
y = dg['dischargestatus']
X = X.fillna(X.mean())
an = training(X,y)
a = mean_confidence_interval(an[0])
t = mean_confidence_interval(an[1])

/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


Only one class present in y_true. ROC AUC score is not defined in that case. error


/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


Only one class present in y_true. ROC AUC score is not defined in that case. error


/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


In [20]:
from prettytable import PrettyTable
l = [["Fixed" ,f_t, f_a],["Inter ", i_t, i_a],["Cont", c_t, c_a],["Fixed + Inter", fi_t, fi_a],["Fixed + Cont", fc_t, fc_a],["Inter + Cont", ci_t, ci_a],["All", t, a]]

table = PrettyTable(['Parameter', 'Training (Mean Lower Upper)', 'Testing (Mean Lower Upper)'])

for rec in l:
    table.add_row(rec)
    
print(table)

+---------------+-----------------------------+----------------------------+
|   Parameter   | Training (Mean Lower Upper) | Testing (Mean Lower Upper) |
+---------------+-----------------------------+----------------------------+
|     Fixed     |     (0.62, 0.568, 0.672)    |   (0.646, 0.553, 0.739)    |
|     Inter     |     (0.941, 0.91, 0.972)    |    (0.64, 0.568, 0.711)    |
|      Cont     |     (0.89, 0.842, 0.938)    |   (0.539, 0.446, 0.632)    |
| Fixed + Inter |      (0.98, 0.961, 1.0)     |    (0.711, 0.641, 0.78)    |
|  Fixed + Cont |    (0.867, 0.816, 0.918)    |   (0.527, 0.445, 0.608)    |
|  Inter + Cont |    (0.946, 0.913, 0.979)    |    (0.677, 0.604, 0.75)    |
|      All      |     (0.967, 0.945, 0.99)    |   (0.632, 0.526, 0.738)    |
+---------------+-----------------------------+----------------------------+
